In [63]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
titanic = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### 請問資料中有哪些「欄位」你覺得是多餘的，在分析過程中應該被排除的呢？為什麼？
PassengerId：乘客ID，通常是用來標識乘客的唯一值，對生存與否沒有實際意義

Name：乘客姓名，通常也不具有直接的。

Ticket：船票號碼，對於生存與否沒有太的Cabin：客艙號碼，缺失值較多，而且與生存與否的關聯度較低助分析結果的新變數。變數。

In [64]:
titanic.info()
titanic.describe()
df = titanic.copy()
df1 = titanic.copy()
new_df = titanic.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### 針對「有缺失值」的欄位，嘗試不同的處理策略（例如：常數、中位數、平均數填補）比較結果
利用(常數、中位數、平均數)去補df['Age']中缺失值，並將結果平均，發現常數有較高的準確度

In [17]:
# 先將df['Sex']及df['Embarked']固定參數
def trans_sex(x):
    if x == 'male':
        return 1
    return 0
df['sex'] = df['Sex'].apply(trans_sex)

df = pd.get_dummies(df, columns=['Embarked'], dtype='int')
df_train = df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'sex', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]

In [18]:
df_train_med = df_train.copy()
df_train_freq = df_train.copy()
df_train_mean = df_train.copy()

In [19]:
# 常數
freq = df_train_freq.groupby('Age')['Age'].count().sort_values().tail(1).index[0]
df_train_freq['Age'].fillna(freq, inplace=True)

columns_X = list(set(df_train_freq.columns) - {'Survived'})
columns_y = ['Survived']

train_X = df_train_freq[columns_X]
train_y = df_train_freq[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores.mean())

0.7912497646098802


In [20]:
# 中位數
med = df_train_med['Age'].median()
df_train_med['Age'].fillna(med, inplace=True)

columns_X = list(set(df_train_med.columns) - {'Survived'})
columns_y = ['Survived']

train_X = df_train_med[columns_X]
train_y = df_train_med[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores.mean())

0.7890151277383717


In [21]:
# 平均數
mean = df_train_mean['Age'].mean()
df_train_mean['Age'].fillna(mean, inplace=True)

columns_X = list(set(df_train_mean.columns) - {'Survived'})
columns_y = ['Survived']

train_X = df_train_mean[columns_X]
train_y = df_train_mean[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores.mean())

0.789008850668508


### 針對「非數值型」的欄位，嘗試不同的處理策略（例如：LabelEncoder 或 OneHotEncoder）比較結果。
發現利用OneHotEncoder有較高的準確度

In [22]:
df1 = pd.get_dummies(df1, columns=['Sex','Embarked'], dtype='int')
df1_train = df1[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]

In [33]:
#  OneHotEncoder
freq = df1_train.groupby('Age')['Age'].count().sort_values().tail(1).index[0]
df1_train['Age'].fillna(freq, inplace=True)

columns_X = list(set(df1_train.columns) - {'Survived'})
columns_y = ['Survived']

train_X = df1_train[columns_X]
train_y = df1_train[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores.mean())

0.789008850668508


In [30]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

#選擇非數值型的欄位
non_numeric_columns = ['Sex' , 'Embarked']

#使用 LabelEncoder 處理非數值行的欄位
df_label_encoded = titanic.copy()
label_encoder = LabelEncoder()
for column in non_numeric_columns:
  df_label_encoded[column] = label_encoder.fit_transform(df_label_encoded[column])

df2_train = df_label_encoded[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']]

In [32]:
freq = df2_train.groupby('Age')['Age'].count().sort_values().tail(1).index[0]
df2_train['Age'].fillna(freq, inplace=True)

columns_X = list(set(df2_train.columns) - {'Survived'})
columns_y = ['Survived']

train_X = df2_train[columns_X]
train_y = df2_train[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores.mean())

/tmp/ipykernel_12060/1295761589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_train['Age'].fillna(freq, inplace=True)


0.786761659657272


### 請你嘗試不同的特徵工程手法，產生可以幫助分析結果的新變數。

In [66]:
# 把 “父母＋小孩” + “兄弟姊妹＋丈夫妻子” 的總和變成一個新的欄位為家庭大小
new_df['family'] = new_df['Parch'] + new_df['SibSp']

In [67]:
new_df['title'] = new_df['Name'].str.split(', ')
new_df['title'] = new_df['title'].apply(lambda name : name[1].split('.')[0])

In [68]:
Title_Dict = {
    'Mr':'Mr',
    'Mrs':'Mrs',
    'Miss':'Miss',
    'Master':'Master',
    'Don':'Royalty',
    'Rev':'Officer',
    'Dr':'Officer',
    'Mme':'Mrs',
    'Ms':'Mrs',
    'Major':'Officer',
    'Lady':'Royalty',
    'Sir':'Royalty',
    'Mlle':'Miss',
    'Col':'Officer',
    'Capt':'Officer',
    'the Countess':'Royalty',
    'Jonkheer':'Royalty'
}
new_df['title'] = new_df['title'].map(Title_Dict)

In [69]:
new_df =  pd.get_dummies(new_df, columns=['Sex','Embarked','title'], dtype='int')

In [70]:
df_new_train = new_df[['Survived', 'Pclass', 'Age', 'family', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
                      'title_Master', 'title_Miss', 'title_Mr', 'title_Mrs', 'title_Officer', 'title_Royalty']]

In [71]:
freq = df_new_train.groupby('Age')['Age'].count().sort_values().tail(1).index[0]
df_new_train['Age'].fillna(freq, inplace=True)

columns_X = list(set(df_new_train.columns) - {'Survived'})
columns_y = ['Survived']

train_X = df_new_train[columns_X]
train_y = df_new_train[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

/tmp/ipykernel_116/3189798822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_train['Age'].fillna(freq, inplace=True)


[0.83240223 0.8258427  0.80337079 0.80337079 0.85955056]
